In [14]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [16]:
coin_list = ['BTC', 'ETH', 'LTC','BNB','USDC']


In [17]:
main_df = pd.DataFrame()


In [24]:
for coin in coin_list:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])

    # Defining the Start Date and End Date
    datetime_end = datetime(2023, 7, 1, 0, 0)
    datetime_check = datetime(2023, 6, 1, 0, 0)

    while len(df) > 0:
        if datetime_end == datetime_check:
            break

        datetime_start = datetime_end - relativedelta(days= 1)

        #Api untuk scrapping
        url = 'https://production.api.coindesk.com/v2/price/values/'+ coin +'?start_date='+datetime_start.strftime("%Y-%m-%dT%H:%M") + '&end_date=' + datetime_end.strftime("%Y-%m-%dT%H:%M") + '&ohlc=true'

        #Menggunakan request u/ ambil data dari API dalam format JSON dan simpan ke dalam DataFrame.
        temp_data = requests.get(url).json()
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']

        #Menghapus kolom Timestamp
        df = df.drop(['Timestamp'], axis=1)

        #Menampilkan selisih waktu permenit
        # df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]

        #Menampilkan selisih waktu per-15mnt
        interval = 15
        df['Datetime'] = [datetime_end - relativedelta(minutes=i*interval) for i in range(0, len(df))]

        coin_df = df.append(coin_df)
        datetime_end = datetime_start

    coin_df['Symbol'] = coin
    main_df = main_df.append(coin_df)

<ipython-input-24-5473b5dde62e>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append(coin_df)
<ipython-input-24-5473b5dde62e>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append(coin_df)
<ipython-input-24-5473b5dde62e>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append(coin_df)
<ipython-input-24-5473b5dde62e>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append(coin_df)
<ipython-input-24-5473b5dde62e>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append(coin_df)


In [38]:
main_df

,Open,High,Low,Close,Datetime,Symbol
0,30474.138070,30504.304554,30441.645957,30444.894265,2023-07-01 22:24:00,BTC
1,30437.444042,30501.731123,30433.148885,30501.716426,2023-07-01 22:25:00,BTC
2,30508.623752,30525.028258,30485.033311,30512.312951,2023-07-01 22:26:00,BTC
3,30516.308231,30516.308231,30456.465415,30470.487261,2023-07-01 22:27:00,BTC
4,30469.198413,30484.565507,30407.434889,30466.633700,2023-07-01 22:28:00,BTC
...,...,...,...,...,...,...
91,0.999879,1.000148,0.999800,1.000107,2023-06-30 01:15:00,USDC
92,1.000014,1.000166,0.999917,0.999994,2023-06-30 01:00:00,USDC
93,0.999945,1.000149,0.999873,1.000082,2023-06-30 00:45:00,USDC
94,0.999981,1.000164,0.999913,1.000050,2023-06-30 00:30:00,USDC


In [26]:
main_df.dtypes

Open               float64
High               float64
Low                float64
Close              float64
Datetime    datetime64[ns]
Symbol              object
dtype: object

In [27]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29760 entries, 0 to 95
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Open      29760 non-null  float64       
 1   High      29760 non-null  float64       
 2   Low       29760 non-null  float64       
 3   Close     29760 non-null  float64       
 4   Datetime  29760 non-null  datetime64[ns]
 5   Symbol    29760 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.6+ MB


In [34]:
unique_datatime = main_df['Open'].unique()
print(unique_datatime)

[3.04741381e+04 3.04374440e+04 3.05086238e+04 ... 9.99944831e-01
 9.99981282e-01 1.00007869e+00]


In [31]:
cryptocurrency_df = pd.DataFrame(main_df['Close'].values.reshape(len(coin_list), -1).transpose(), index=main_df['Datetime'][:int(len(main_df) / len(coin_list))], columns=coin_list)
cryptocurrency_df.to_csv('cryptocurrency_df.csv')

In [35]:
df=pd.read_csv('cryptocurrency_df.csv')

In [36]:
df.shape

(5952, 6)

In [37]:
df

,Datetime,BTC,ETH,LTC,BNB,USDC
0,2023-07-01 22:24:00,30444.894265,1878.508866,237.194728,30240.601314,83.360355
1,2023-07-01 22:25:00,30501.716426,1877.691919,236.335663,30272.260066,83.372196
2,2023-07-01 22:26:00,30512.312951,1877.392261,236.159960,30316.925880,83.357224
3,2023-07-01 22:27:00,30470.487261,1876.647380,236.029398,30359.613194,83.158948
4,2023-07-01 22:28:00,30466.633700,1879.860387,236.910927,30313.690141,83.436857
...,...,...,...,...,...,...
5947,2023-06-22 23:55:00,1870.789114,236.230787,30212.846363,83.161404,1.000107
5948,2023-06-22 23:56:00,1872.192178,236.435313,30210.769995,83.500986,0.999994
5949,2023-06-22 23:57:00,1875.054872,236.816032,30218.973641,83.370040,1.000082
5950,2023-06-22 23:58:00,1874.546448,236.261061,30222.985397,83.071796,1.000050
